In [15]:
# !pip install -r ./requirements.txt

In [2]:
# !pip uninstall virny -y

In [4]:
# Install using an HTTP link
# !pip install git+https://github.com/DataResponsibly/Virny.git@feature/prepare_for_uncertainty_experiments

# Install using an SSH link
# !pip install git+ssh://git@github.com/DataResponsibly/Virny.git@feature/prepare_for_uncertainty_experiments

In [5]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [6]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [7]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../../../..")

print('Current location: ', os.getcwd())

Current location:  /Users/denys_herasymuk/UCU/4course_2term/Bachelor_Thesis/Code/fairness-variance


## Import dependencies

In [8]:
import copy

from virny.utils.custom_initializers import create_config_obj
from virny.datasets import ACSIncomeDataset

from configs.constants import TEST_SET_FRACTION, EXPERIMENT_SEEDS
from configs.models_config_for_tuning import get_folktables_employment_models_params_for_tuning

from source.experiment_interface import run_exp_iter_with_eq_odds

pip install 'aif360[LawSchoolGPA]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'


## Define Input Variables

In [9]:
ROOT_DIR = os.getcwd()
DATASET_NAME = 'ACSIncomeDataset'
FAIRNESS_INTERVENTION_NAME = 'ROC'
EXPERIMENT_NAME = 'ROC_acs_income'
DB_COLLECTION_NAME = 'one_repair_lvl_many_models'
FAIR_INTERVENTION_PARAMS_LST = [True]
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                     FAIRNESS_INTERVENTION_NAME, EXPERIMENT_NAME)

config_yaml_path = os.path.join(ROOT_DIR, 'notebooks', 'diff_fairness_interventions_exp',
                                FAIRNESS_INTERVENTION_NAME, EXPERIMENT_NAME, 'folk_GA_2018_config.yaml')
metrics_computation_config = create_config_obj(config_yaml_path=config_yaml_path)

## Define a db writer and custom fields to insert into your database

In [10]:
import os
from dotenv import load_dotenv

load_dotenv('./configs/secrets.env')
os.getenv("DB_NAME")

'fairness_variance'

In [11]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

In [12]:
import uuid

custom_table_fields_dct = {
    'session_uuid': str(uuid.uuid4()),
    # 'session_uuid': 'bec5d2ec-0722-484c-b70b-a0206f680965',
}
print('Current session uuid: ', custom_table_fields_dct['session_uuid'])

Current session uuid:  bec5d2ec-0722-484c-b70b-a0206f680965


## Initialize custom objects

In [13]:
data_loader = ACSIncomeDataset(state=['GA'], year=2018, with_nulls=False,
                               subsample_size=15_000, subsample_seed=42)
data_loader.X_data.head()

,SCHL,COW,MAR,OCCP,POBP,RELP,SEX,RAC1P,AGEP,WKHP
0,23,7,3,230,36,0,1,1,55,55.0
1,16,1,5,4110,13,2,2,1,20,35.0
2,16,4,3,4130,51,0,2,1,59,30.0
3,18,4,1,4020,13,0,1,2,43,40.0
4,14,1,1,8300,20,1,2,2,33,20.0


In [14]:
data_loader.X_data.shape

(3000, 10)

## Run experiment iterations

### Experiment iteration 1

In [15]:
# tuned_params_filenames = [
#     'tuning_results_Folktables_GA_2018_Income_20240104__100612.csv'
# ]
# tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
#                                       FAIRNESS_INTERVENTION_NAME, EXPERIMENT_NAME, tuned_params_filename)
#                          for tuned_params_filename in tuned_params_filenames]

In [16]:
# Configs for an experiment iteration
exp_iter_num = 1
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
models_params_for_tuning = {
    'LGBMClassifier': models_params_for_tuning['LGBMClassifier']
}

In [17]:
run_exp_iter_with_eq_odds(data_loader=exp_iter_data_loader,
                          experiment_seed=experiment_seed,
                          test_set_fraction=TEST_SET_FRACTION,
                          db_writer_func=db_writer_func,
                          fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                          models_params_for_tuning=models_params_for_tuning,
                          metrics_computation_config=metrics_computation_config,
                          custom_table_fields_dct=custom_table_fields_dct,
                          with_tuning=True,
                          # with_tuning=False,
                          # tuned_params_df_paths=tuned_params_df_paths,
                          save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                          dataset_name=DATASET_NAME,
                          postprocessor_name='ROC',
                          verbose=True)

2024-01-05 17:46:18 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:


{'dataset_split_seed': 100,
 'experiment_iteration': 'Exp_iter_1',
 'fair_intervention_params_lst': '[True]',
 'model_init_seed': 100,
 'session_uuid': 'bec5d2ec-0722-484c-b70b-a0206f680965'}


data_loader.X_data.head(20)


,SCHL,COW,MAR,OCCP,POBP,RELP,SEX,RAC1P,AGEP,WKHP,SEX&RAC1P_binary
0,23,7,3,230,36,0,1,1,55,55.0,1
1,16,1,5,4110,13,2,2,1,20,35.0,1
2,16,4,3,4130,51,0,2,1,59,30.0,1
3,18,4,1,4020,13,0,1,2,43,40.0,1
4,14,1,1,8300,20,1,2,2,33,20.0,0
5,16,1,1,6355,13,1,1,1,37,40.0,1
6,22,1,1,120,13,1,1,2,38,55.0,1
7,19,6,1,2600,47,1,1,1,36,50.0,1
8,18,3,1,5400,13,0,2,1,50,40.0,1
9,21,1,5,2710,12,13,2,1,36,40.0,1


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-05 17:46:18 experiment_interface.py INFO    : The dataset is preprocessed


intervention_option:  True
Using ROC postprocessor
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__SCHL_1', 'cat__SCHL_10', 'cat__SCHL_11', 'cat__SCHL_12',
       'cat__SCHL_13', 'cat__SCHL_14', 'cat__SCHL_15', 'cat__SCHL_16',
       'cat__SCHL_17', 'cat__SCHL_18',
       ...
       'cat__RELP_3', 'cat__RELP_4', 'cat__RELP_5', 'cat__RELP_6',
       'cat__RELP_7', 'cat__RELP_8', 'cat__RELP_9', 'num__AGEP', 'num__WKHP',
       'SEX&RAC1P_binary'],
      dtype='object', length=536)
Top indexes of an X_test in the current base flow dataset:  Int64Index([1273, 1380,  356, 1920,  561,  105,  999, 2563,  584,   31,  610,
            1856, 2740, 1804, 2691, 1292, 2305,  221,  291, 1290],
           dtype='int64')
Top indexes of an y_test in the current base flow dataset:  Int64Index([1273, 1380,  356, 1920,  561,  105,  999, 2563,  584,   31,  610,
            1856, 2740, 1804, 2691, 1292, 2305,  221,  291, 1290],
           dtype='int64')
2024/01/05, 17:46:18: Tuning LogisticRegressi

2024-01-05 17:46:23 experiment_interface.py INFO    : Models are tuned and saved to a file


Analyze multiple models:   0%|          | 0/1 [00:00<?, ?it/s]

Enabled a postprocessing mode


Classifiers testing by bootstrap:   0%|          | 0/50 [00:00<?, ?it/s]

### Experiment iteration 2

In [11]:
# Configs for an experiment iteration
exp_iter_num = 2
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                      FAIRNESS_INTERVENTION_NAME, EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
models_params_for_tuning = {
    'LGBMClassifier': models_params_for_tuning['LGBMClassifier']
}

In [12]:
run_exp_iter_with_eq_odds(data_loader=exp_iter_data_loader,
                          experiment_seed=experiment_seed,
                          test_set_fraction=TEST_SET_FRACTION,
                          db_writer_func=db_writer_func,
                          fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                          models_params_for_tuning=models_params_for_tuning,
                          metrics_computation_config=metrics_computation_config,
                          custom_table_fields_dct=custom_table_fields_dct,
                          # with_tuning=True,
                          with_tuning=False,
                          tuned_params_df_paths=tuned_params_df_paths,
                          save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                          dataset_name=DATASET_NAME,
                          postprocessor_name='ROC',
                          verbose=True)

2024-01-04 09:04:59 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 200,
 'experiment_iteration': 'Exp_iter_2',
 'fair_intervention_params_lst': '[True]',
 'model_init_seed': 200,
 'session_uuid': '3976c1bf-99fa-49f4-97ec-44b13cb64ef4'}


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

intervention_options:  True


2024-01-04 09:04:59 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-04 09:04:59 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


...

Enabled a postprocessing mode


Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

### Experiment iteration 3

In [11]:
# Configs for an experiment iteration
exp_iter_num = 3
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                      FAIRNESS_INTERVENTION_NAME, EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
models_params_for_tuning = {
    'LGBMClassifier': models_params_for_tuning['LGBMClassifier']
}

In [ ]:
run_exp_iter_with_eq_odds(data_loader=exp_iter_data_loader,
                          experiment_seed=experiment_seed,
                          test_set_fraction=TEST_SET_FRACTION,
                          db_writer_func=db_writer_func,
                          fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                          models_params_for_tuning=models_params_for_tuning,
                          metrics_computation_config=metrics_computation_config,
                          custom_table_fields_dct=custom_table_fields_dct,
                          # with_tuning=True,
                          with_tuning=False,
                          tuned_params_df_paths=tuned_params_df_paths,
                          save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                          dataset_name=DATASET_NAME,
                          postprocessor_name='ROC',
                          verbose=True)

2024-01-04 09:47:45 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 300,
 'experiment_iteration': 'Exp_iter_3',
 'fair_intervention_params_lst': '[True]',
 'model_init_seed': 300,
 'session_uuid': '3976c1bf-99fa-49f4-97ec-44b13cb64ef4'}


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

intervention_options:  True


2024-01-04 09:47:45 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-04 09:47:45 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


...

Enabled a postprocessing mode


Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

### Experiment iteration 4

In [25]:
# Configs for an experiment iteration
exp_iter_num = 4
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                      FAIRNESS_INTERVENTION_NAME, EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
models_params_for_tuning = {
    'LGBMClassifier': models_params_for_tuning['LGBMClassifier']
}

In [3]:
run_exp_iter_with_eq_odds(data_loader=exp_iter_data_loader,
                          experiment_seed=experiment_seed,
                          test_set_fraction=TEST_SET_FRACTION,
                          db_writer_func=db_writer_func,
                          fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                          models_params_for_tuning=models_params_for_tuning,
                          metrics_computation_config=metrics_computation_config,
                          custom_table_fields_dct=custom_table_fields_dct,
                          # with_tuning=True,
                          with_tuning=False,
                          tuned_params_df_paths=tuned_params_df_paths,
                          save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                          dataset_name=DATASET_NAME,
                          postprocessor_name='ROC',
                          verbose=True)

### Experiment iteration 5

In [27]:
# Configs for an experiment iteration
exp_iter_num = 5
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                      FAIRNESS_INTERVENTION_NAME, EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
models_params_for_tuning = {
    'LGBMClassifier': models_params_for_tuning['LGBMClassifier']
}

In [4]:
run_exp_iter_with_eq_odds(data_loader=exp_iter_data_loader,
                          experiment_seed=experiment_seed,
                          test_set_fraction=TEST_SET_FRACTION,
                          db_writer_func=db_writer_func,
                          fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                          models_params_for_tuning=models_params_for_tuning,
                          metrics_computation_config=metrics_computation_config,
                          custom_table_fields_dct=custom_table_fields_dct,
                          # with_tuning=True,
                          with_tuning=False,
                          tuned_params_df_paths=tuned_params_df_paths,
                          save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                          dataset_name=DATASET_NAME,
                          postprocessor_name='ROC',
                          verbose=True)

### Experiment iteration 6

In [11]:
# Configs for an experiment iteration
exp_iter_num = 6
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                      FAIRNESS_INTERVENTION_NAME, EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
models_params_for_tuning = {
    'LGBMClassifier': models_params_for_tuning['LGBMClassifier']
}

In [12]:
run_exp_iter_with_eq_odds(data_loader=exp_iter_data_loader,
                          experiment_seed=experiment_seed,
                          test_set_fraction=TEST_SET_FRACTION,
                          db_writer_func=db_writer_func,
                          fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                          models_params_for_tuning=models_params_for_tuning,
                          metrics_computation_config=metrics_computation_config,
                          custom_table_fields_dct=custom_table_fields_dct,
                          # with_tuning=True,
                          with_tuning=False,
                          tuned_params_df_paths=tuned_params_df_paths,
                          save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                          dataset_name=DATASET_NAME,
                          postprocessor_name='ROC',
                          verbose=True)

2024-01-04 09:55:19 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 600,
 'experiment_iteration': 'Exp_iter_6',
 'fair_intervention_params_lst': '[True]',
 'model_init_seed': 600,
 'session_uuid': '3976c1bf-99fa-49f4-97ec-44b13cb64ef4'}


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

intervention_options:  True


2024-01-04 09:55:19 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-04 09:55:19 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


...

Enabled a postprocessing mode


Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB